## ANN Regression Practical Implementation

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [3]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## Preprocess the data
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [5]:
## Define the label encoder
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [6]:
# One-hot encoder for the geography
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
## Combining the one hot encoded geography to original data
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [8]:
# Split the data in features and target
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [9]:
# Split the data in training and testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Standard Scaler
from sklearn.preprocessing import StandardScaler, LabelEncoder
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
#  Save the Encoders and scaler for user
with open('label_encoder_gender.pkl', 'wb') as file:
        pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
        pickle.dump(onehot_encoder_geo, file)

with open('scaler.pkl', 'wb') as file:
        pickle.dump(scaler, file)

### ANN Regression Problem Statement

In [12]:
X_train.shape[1],

(12,)

In [13]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)


X_train shape: (8000, 12)
y_train shape: (8000,)
X_test shape: (2000, 12)
y_test shape: (2000,)


In [14]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
# Setup of tensorboard
log_dir="regressionlogs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=TensorBoard(log_dir=log_dir, histogram_freq=1)

In [15]:
## setup Early Stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [23]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# Step 2: Ensure data is in correct format
X_train = np.array(X_train, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
y_train = np.array(y_train, dtype=np.float32).reshape(-1, 1)
y_test = np.array(y_test, dtype=np.float32).reshape(-1, 1)

# Step 3: Clear any previous TensorFlow models
tf.keras.backend.clear_session()

# Step 4: Build the model with correct layer names
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],), name='InputLayer'),
    Dense(32, activation='relu', name='HiddenLayer1'),
    Dense(1, name='OutputLayer')
])

model.summary()

# Step 5: Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

# Step 6: Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, callbacks=[early_stopping_callback, tensorboard_callback])


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ InputLayer (Dense)              │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ HiddenLayer1 (Dense)            │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ OutputLayer (Dense)             │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 99242.6875 - mae: 99242.6875 - val_loss: 98534.8047 - val_mae: 98534.8047
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 100368.4531 - mae: 100368.4531 - val_loss: 97209.7266 - val_mae: 97209.7266
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 97883.6328 - mae: 97883.6328 - val_loss: 93846.1719 - val_mae: 93846.1719
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 95333.0312 - mae: 95333.0312 - val_loss: 88149.9922 - val_mae: 88149.9922
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 87990.0859 - mae: 87990.0859 - val_loss: 80606.9375 - val_mae: 80606.9375
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 79399.8906 - mae: 79399.8906 - val_loss: 72308.5078 - val_mae: 72308.5078
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 72045.6328 - mae: 72045.6328 - val_loss: 64667.2148 - val_mae: 64667.2148
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step 

In [24]:
%load_ext tensorboard 

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [31]:
%tensorboard --logdir regressionlogs/fit20250403-233658

Reusing TensorBoard on port 6007 (pid 8780), started 0:00:02 ago. (Use '!kill 8780' to kill it.)

In [40]:
## Evaluate the model on test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test_MAE {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 50969.0117 - mae: 50969.0117
Test_MAE 50153.25390625


In [41]:
model.save('regression.h5')

In [1]:
import numpy as np
print(np.__version__)


2.1.3


###  By increasing of layers the model  learns better

In [33]:
from tensorflow.keras.layers import LeakyReLU

model = Sequential([
    Dense(128, input_shape=(X_train.shape[1],), name='InputLayer'),
    LeakyReLU(alpha=0.1),
    
    Dense(64, name='FirstHiddenLayer'),
    LeakyReLU(alpha=0.1),
    
    Dense(32,name='SecondHiddenLayer'), 
    LeakyReLU(alpha=0.1),
    
    Dense(1, name='OutputLayer')
])


In [35]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ InputLayer (Dense)              │ (None, 128)            │         1,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_3 (LeakyReLU)       │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ FirstHiddenLayer (Dense)        │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_4 (LeakyReLU)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SecondHiddenLayer (Dense)       │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_5 (LeakyReLU)       │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ OutputLayer (Dense)             │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,033 (47.00 KB)

 Trainable params: 12,033 (47.00 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,  
    batch_size=32,
    callbacks=[early_stop]
)


Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - loss: 100237.2266 - mae: 100237.2266 - val_loss: 88221.5781 - val_mae: 88221.5781
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 77528.0156 - mae: 77528.0156 - val_loss: 50491.3242 - val_mae: 50491.3242
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 50635.2539 - mae: 50635.2539 - val_loss: 50211.6523 - val_mae: 50211.6523
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 49751.7227 - mae: 49751.7227 - val_loss: 50215.3555 - val_mae: 50215.3555
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 49377.6992 - mae: 49377.6992 - val_loss: 50202.4102 - val_mae: 50202.4102
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 49845.5312 - mae: 49845.5312 - val_loss: 50223.8555 - val_mae: 50223.8555
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - loss: 49600.5078 - mae: 49600.5078 - val_loss: 50195.4375 - val_mae: 50195.4375
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step 